In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import json
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
df_bbidding = pd.read_csv('bidding_training.csv')
df_bidding_imbalance_price = pd.read_csv('bidding_training_predictions_imbalance_price.csv')
df_bidding_imbalance_price = df_bidding_imbalance_price[["timestamp_utc","imbalance_price_predictions"]]
df_bidding_imbalance_price.timestamp_utc = pd.to_datetime(df_bidding_imbalance_price.timestamp_utc)
df_bidding_day_ahead_price = pd.read_csv('bidding_training_predictions_day_ahead_price.csv')
df_bidding_day_ahead_price = df_bidding_day_ahead_price[["timestamp_utc","day_ahead_price_predictions"]]
df_bidding_day_ahead_price.timestamp_utc = pd.to_datetime(df_bidding_day_ahead_price.timestamp_utc)
df_bbidding["day_ahead_price"] = df_bbidding["price_x"].rename("day_ahead_price")
df_bbidding["market_price"] = df_bbidding["price_y"].rename("market_price")
# df_day_ahead = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/day_ahead_price.csv')
# df_imbalance = pd.read_csv('D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/imbalance_price.csv')

In [3]:
df_bbidding.timestamp_utc = pd.to_datetime(df_bbidding.timestamp_utc)
df_bbidding = df_bbidding.merge(df_bidding_imbalance_price, on="timestamp_utc")
df_bbidding = df_bbidding.merge(df_bidding_day_ahead_price, on="timestamp_utc")

In [5]:
df_bbidding["price_difference"] = df_bbidding["day_ahead_price"] - df_bbidding["imbalance_price"]

In [6]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

# Define the revenue function (objective function)
def revenue(zb, Target_MW, price_diff):
    return price_diff * zb + -0.07*(Target_MW-zb)**2

# Negative revenue function (for minimization)
def negative_revenue(zb, Target_MW, price_difference):
    return -revenue(zb, Target_MW, price_difference)

# Optimization function to compute the optimal bidding value for each row
def optimize_bidding(row):
    # Extract the values from the row
    Target_MW = row['Target_MW']
    price_difference = row['price_difference']
    
    # Initial guess for zb (midpoint between 0 and Target_MW)
    initial_zb = Target_MW / 2
    
    # Bounds for zb (as per KKT conditions)
    bounds = [(0, 1800)]
    
    # Perform the optimization
    result = minimize(negative_revenue, initial_zb, args=(Target_MW, price_difference), bounds=bounds)
    
    # Optimal trade value (zb)
    return result.x[0]

# Apply the optimization to each row and replace column '5' with the optimized trade value
df_bbidding['optimized_trade'] = df_bbidding.apply(optimize_bidding, axis=1)

# Now calculate the revenue using the optimized trade values
df_bbidding['revenue_optimal'] = df_bbidding['day_ahead_price'] * df_bbidding['optimized_trade'] + \
                         (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']) * \
                         (df_bbidding['imbalance_price'] - 0.07 * (df_bbidding['Target_MW'] - df_bbidding['optimized_trade']))



In [7]:
df_bbidding.revenue_optimal.mean()

56847.65672592287